In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import os
import random
import onnx  # pip install onnx
import onnxruntime as ort  # pip install onnxruntime
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = './dataset/data_v2_preprocessed'
CONTROLLER_PATH = "./super_resolution_kinda_safe.onnx"

class DataLoader(object):
    def __init__(self):
        self.trajectories = []
        for fp, data in self.get_raw_data():
            positions = [0]
            for speed in np.array(data['Velocity'])[:-1] / 3.6:
                positions.append(positions[-1] + 0.1 * speed)
            self.trajectories.append({
                'path': fp,
                'timestep': round(data['Time'][1] - data['Time'][0], 3),
                'duration': round(data['Time'].max() - data['Time'].min(), 3),
                'size': len(data['Time']),
                'times': np.array(data['Time']) - data['Time'][0],
                'positions': positions,  # np.array(data['DistanceGPS']),
                'velocities': np.array(data['Velocity']) / 3.6,
                'accelerations': np.array(data['Acceleration'])
            })

    def get_raw_data(self):
        file_paths = list(Path(os.path.join(DATA_PATH)).glob('**/*.csv'))
        data = map(pd.read_csv, file_paths)
        return zip(file_paths, data)

    def get_all_trajectories(self):
        random.shuffle(self.trajectories)
        return iter(self.trajectories)

    def get_trajectories(self, chunk_size=None, count=None):
        for _ in counter(count):
            traj = random.sample(self.trajectories, k=1)[0]
            if chunk_size is None:
                yield dict(traj)
            start_idx = random.randint(0, traj['size'] - chunk_size)
            traj_chunk = {
                k: traj[k][start_idx:start_idx+chunk_size]
                for k in ['times', 'positions', 'velocities', 'accelerations']
            }
            traj_chunk.update({
                'path': traj['path'],
                'timestep': traj['timestep'],
                'duration': round(np.max(traj_chunk['times']) - np.min(traj_chunk['times']), 3),
                'size': len(traj_chunk['times']),
            })
            yield traj_chunk

def get_accel(state):
    # state is [av speed, leader speed, headway] (no normalization needed)
    # output is instant acceleration to apply to the AV
    data = np.array([state]).astype(np.float32)
    outputs = ort_session.run(None, {ort_session.get_inputs()[0].name: data})
    return outputs[0][0][0]

In [4]:
# load i24 trajectory data
data = DataLoader()
print(list(data.get_raw_data()))

trajectories = data.get_all_trajectories()
print(trajectories)

# for traj in trajectories:

#     print(traj.keys())

#     # load RL controller
#     model = onnx.load_model(CONTROLLER_PATH)
#     ort_session = ort.InferenceSession(CONTROLLER_PATH)

#     # initialize AV
#     av_positions = [traj['positions'][0] - 50.0]  # set initial headway
#     av_speeds = [traj['velocities'][0]]  # set initial speed

#     # run AV behind trajectory
#     dt = traj['timestep']
#     print(f'dt={dt}s')
#     for time, leader_pos, leader_speed in zip(traj['times'], traj['positions'], traj['velocities']):
#         # get AV accel
#         av_pos = av_positions[-1]
#         av_speed = av_speeds[-1]
#         av_space_gap = leader_pos - av_pos
#         av_accel = get_accel([av_speed, leader_speed, av_space_gap])

#         # update AV
#         new_av_speed = av_speed + dt * av_accel
#         new_av_pos = av_pos + dt * new_av_speed
#         av_speeds.append(new_av_speed)
#         av_positions.append(new_av_pos)

#     print(f'Trajectory ending at time t={round(time, 1)}s')

#     plt.figure()
#     plt.plot(traj['times'], av_speeds[1:], label='AV')
#     plt.plot(traj['times'], traj['velocities'], label='leader')
#     plt.xlabel('Time (s)')
#     plt.ylabel('Speed (m/s)')
#     plt.legend()
#     plt.show()

[(PosixPath('dataset/data_v2_preprocessed/2021-03-24-21-34-31_2T3MWRFVXLW056972_masterArray_0_5521.csv'),       index          Time   Velocity  Acceleration   SpaceGap  \
0      6382  1.616625e+09  65.708628     -0.524803  41.712756   
1      6383  1.616625e+09  65.590000     -0.524237  41.763663   
2      6384  1.616625e+09  65.457821     -0.522526  41.840866   
3      6385  1.616625e+09  65.323817     -0.519474  41.860908   
4      6386  1.616625e+09  65.225301     -0.514837  41.881897   
5      6387  1.616625e+09  65.106113     -0.508347  41.921914   
6      6388  1.616625e+09  64.998933     -0.499724  41.962315   
7      6389  1.616625e+09  64.858308     -0.488698  42.010268   
8      6390  1.616625e+09  64.797954     -0.475025  42.058220   
9      6391  1.616625e+09  64.668698     -0.458510  42.112745   
10     6392  1.616625e+09  64.535375     -0.439018  42.172738   
11     6393  1.616625e+09  64.411133     -0.416494  42.243630   
12     6394  1.616625e+09  64.288275     -0.39096